In [1]:
import pandas as pd
import numpy as np
import warnings
import math
from statsmodels.stats.stattools import medcouple 

warnings.filterwarnings('ignore') 

In [8]:
df = pd.read_csv("../../all_csvs/abalone.csv")

In [9]:
df.skew(axis = 0, skipna = True)

Length           -0.639873
Diameter         -0.609198
Height            3.128817
Whole weight      0.530959
Shucked weight    0.719098
Viscera weight    0.591852
Shell weight      0.620927
Rings             1.114102
dtype: float64

In [10]:
for col in df.columns:
    if df[col].dtypes == 'float64' or df[col].dtypes == 'int64' :
        pass
    else:
        df = df.drop( columns = [col])

In [11]:
def outliers_position(ys):
    q1, q3 = np.percentile(ys, [25, 75])
    iqr = q3 - q1
    mc = medcouple(ys)
    print("Mc: ", mc)
    if mc >= 0:
        print("maior igual")
        lower_bound = q1 - (1.5 * math.exp(-4.0 * mc) * iqr) 
        upper_bound = q3 + (1.5 * math.exp(3 * mc) * iqr)
    else:
        print("menor")
        lower_bound = q1 - (1.5 * math.exp(-3.0 * mc) * iqr) 
        upper_bound = q3 + (1.5 * math.exp(4 * mc) * iqr)
    return np.where((ys > upper_bound) | (ys < lower_bound))

In [12]:
def outliers_value(ys):
    aux = []
    q1, q3 = np.percentile(ys, [25, 75])
    iqr = q3 - q1
    mc = medcouple(ys)
    print("Mc: ", mc)
    if mc >= 0:
        print("maior igual")
        lower_bound = q1 - (1.5 * math.exp(-4.0 * mc) * iqr) 
        upper_bound = q3 + (1.5 * math.exp(3 * mc) * iqr)
    else:
        print("menor")
        lower_bound = q1 - (1.5 * math.exp(-3.0 * mc) * iqr) 
        upper_bound = q3 + (1.5 * math.exp(4 * mc) * iqr)
    for y in ys:
        if (y > upper_bound) or (y < lower_bound):
            aux.append(y)
    return aux

In [13]:
out_posi = len(df.columns)*[[]]

for col,i in zip(df.columns,range(len(df.columns))):
    out_posi[i] = outliers_position(df[col].values)[0]

Mc:  -0.1818181818181822
menor
Mc:  -0.1724137931034481
menor
Mc:  -0.043478260869565376
menor
Mc:  0.023851076207097234
maior igual
Mc:  0.08262108262108253
maior igual
Mc:  0.06410256410256399
maior igual
Mc:  -0.017142857142856998
menor
Mc:  0.25
maior igual


In [14]:
len(out_posi[3])

19

In [15]:
outlier_values = len(df.columns)*[[]]
for col,i in zip(df.columns,range(len(out_posi))):
    outlier_values[i] = outliers_value(df[col].values)

Mc:  -0.1818181818181822
menor
Mc:  -0.1724137931034481
menor
Mc:  -0.043478260869565376
menor
Mc:  0.023851076207097234
maior igual
Mc:  0.08262108262108253
maior igual
Mc:  0.06410256410256399
maior igual
Mc:  -0.017142857142856998
menor
Mc:  0.25
maior igual


In [16]:
my_locations = list(df.columns)
outlier_value = list(df.columns)

In [17]:
for i in range(len(outlier_value)):
    outlier_value[i] = outlier_value[i] + " outlier candidate"

for i in range(len(my_locations)):
    my_locations[i] = my_locations[i] + " line"

In [18]:
print(my_locations)
print("\n")
print(outlier_value)

['Length line', 'Diameter line', 'Height line', 'Whole weight line', 'Shucked weight line', 'Viscera weight line', 'Shell weight line', 'Rings line']


['Length outlier candidate', 'Diameter outlier candidate', 'Height outlier candidate', 'Whole weight outlier candidate', 'Shucked weight outlier candidate', 'Viscera weight outlier candidate', 'Shell weight outlier candidate', 'Rings outlier candidate']


In [19]:
new_df = pd.DataFrame(out_posi)
new_df = new_df.T
new_df.columns = my_locations

In [20]:
new_df

,Length line,Diameter line,Height line,Whole weight line,Shucked weight line,Viscera weight line,Shell weight line,Rings line
0,334.0,334.0,164.0,165.0,891.0,1204.0,81.0,42.0
1,358.0,355.0,236.0,358.0,1051.0,1422.0,129.0,43.0
2,478.0,358.0,277.0,891.0,1207.0,1762.0,157.0,44.0
3,1051.0,372.0,307.0,1051.0,1209.0,1763.0,163.0,48.0
4,1052.0,478.0,506.0,1052.0,1426.0,2334.0,164.0,58.0
...,...,...,...,...,...,...,...,...
479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4119.0
480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4149.0
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4151.0
482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4154.0


In [21]:
new_df2 = pd.DataFrame(outlier_values)
new_df2 = new_df2.T
new_df2.columns = outlier_value

In [22]:
new_df2

,Length outlier candidate,Diameter outlier candidate,Height outlier candidate,Whole weight outlier candidate,Shucked weight outlier candidate,Viscera weight outlier candidate,Shell weight outlier candidate,Rings outlier candidate
0,0.740,0.600,0.230,2.5500,1.1465,0.5500,0.675,5.0
1,0.745,0.580,0.010,2.4990,1.1335,0.5640,0.780,5.0
2,0.735,0.585,0.240,2.8255,1.1965,0.6415,0.635,4.0
3,0.735,0.580,0.240,2.5550,1.4880,0.7600,1.005,6.0
4,0.765,0.590,0.235,2.3020,1.1565,0.5900,0.815,4.0
...,...,...,...,...,...,...,...,...
479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0


In [23]:
for col in new_df.columns:
    new_df2[col] = new_df[col]

In [24]:
col = list(new_df2.columns.values)
col.sort()
new_df2 = new_df2[col]
new_df2 = new_df2.replace(np.nan,"")

In [25]:
new_df2.head(60)

,Diameter line,Diameter outlier candidate,Height line,Height outlier candidate,Length line,Length outlier candidate,Rings line,Rings outlier candidate,Shell weight line,Shell weight outlier candidate,Shucked weight line,Shucked weight outlier candidate,Viscera weight line,Viscera weight outlier candidate,Whole weight line,Whole weight outlier candidate
0,334,0.6,164,0.23,334,0.74,42.0,5.0,81,0.675,891,1.1465,1204,0.55,165,2.55
1,355,0.58,236,0.01,358,0.745,43.0,5.0,129,0.78,1051,1.1335,1422,0.564,358,2.499
2,358,0.585,277,0.24,478,0.735,44.0,4.0,157,0.635,1207,1.1965,1762,0.6415,891,2.8255
3,372,0.58,307,0.24,1051,0.735,48.0,6.0,163,1.005,1209,1.488,1763,0.76,1051,2.555
4,478,0.59,506,0.235,1052,0.765,58.0,4.0,164,0.815,1426,1.1565,2334,0.59,1052,2.302
5,480,0.585,694,0.02,1204,0.735,69.0,6.0,165,0.725,1427,1.232,3427,0.575,1207,2.505
6,882,0.58,719,0.025,1205,0.745,83.0,21.0,166,0.85,1528,1.351,3715,0.5745,1209,2.657
7,885,0.585,720,0.025,1206,0.75,123.0,6.0,167,0.65,1754,1.1455,,,1427,2.5085
8,891,0.595,891,0.23,1207,0.755,124.0,6.0,168,0.76,1762,1.1155,,,1762,2.5155
9,1051,0.6,1174,0.015,1208,0.755,125.0,5.0,169,0.62,1763,1.3485,,,1763,2.7795


In [28]:
for col in df.columns:
    temp_df = new_df2.filter(like = col )
    temp_df.to_csv(col+".csv", index = False)

In [29]:
new_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Diameter line                     484 non-null    object 
 1   Diameter outlier candidate        484 non-null    object 
 2   Height line                       484 non-null    object 
 3   Height outlier candidate          484 non-null    object 
 4   Length line                       484 non-null    object 
 5   Length outlier candidate          484 non-null    object 
 6   Rings line                        484 non-null    float64
 7   Rings outlier candidate           484 non-null    float64
 8   Shell weight line                 484 non-null    object 
 9   Shell weight outlier candidate    484 non-null    object 
 10  Shucked weight line               484 non-null    object 
 11  Shucked weight outlier candidate  484 non-null    object 
 12  Viscera 